In [1]:
import pandas as pd
import os, sys, json, math, time

In [2]:
ersa = '/data100t1/home/grahame/projects/compadre/ersa/ersa2_py3/ersa.py'

### Write code that uses the primus-ersa helper functions to 1. call ersa and 2. update the vector file (with python) accordingly

In [54]:
prefix = 'sim64'

test_dir = f'/data100t1/home/grahame/projects/compadre/primus-ersa-v2/output/{prefix}/'
data_dir = '/data100t1/home/grahame/projects/compadre/primus-ersa-v2/example_data/'

vector_file = data_dir + f'{prefix}.genome_KDE_likelihood_vectors'

df = pd.read_csv(vector_file, sep='\t')
df['values'] = df['LIKELIHOOD_VECTOR'].str.split(',')
df['values'] = df['values'].apply(lambda x: [float(val) for val in x])
df = pd.concat([df, df['values'].apply(lambda x: pd.Series(x))], axis=1)
df = df.rename(columns={0: 'p0', 1: 'p1', 2: "p2", 3: "p3", 4: "p4", 5: "p5"})
df = df.drop(columns=['values'])

In [55]:
df

,FID1,IID1,FID2,IID2,MOST_LIKELY_REL,LIKELIHOOD_VECTOR,IBD0,IBD1,IBD2,PI_HAT,-1,POSSIBLE_RELS,LIKELIHOOD_CUTOFF,p0,p1,p2,p3,p4,p5
0,0,id1,0,id2,UN,"0,4.04568136411392e-51,1.2021878588324e-05,0.0...",1.000,0.000,0.000,0.0000,-1,UN,0.3,0.000000e+00,4.045681e-51,0.000012,4.802023e-04,4.091679e-02,0.958591
1,0,id1,0,id3,PC,"0.999488504078663,1.54522754683251e-18,0.00051...",0.000,1.000,0.000,0.5000,-1,PC,0.3,9.994885e-01,1.545228e-18,0.000511,4.009565e-13,4.171423e-27,0.000000
2,0,id1,0,id4,PC,"0.999488504078663,1.54522754683251e-18,0.00051...",0.000,1.000,0.000,0.5000,-1,PC,0.3,9.994885e-01,1.545228e-18,0.000511,4.009565e-13,4.171423e-27,0.000000
3,0,id1,0,id5,PC,"0.999488504078663,1.54522754683251e-18,0.00051...",0.000,1.000,0.000,0.5000,-1,PC,0.3,9.994885e-01,1.545228e-18,0.000511,4.009565e-13,4.171423e-27,0.000000
4,0,id1,0,id6,UN,"0,4.04568136411392e-51,1.2021878588324e-05,0.0...",1.000,0.000,0.000,0.0000,-1,UN,0.3,0.000000e+00,4.045681e-51,0.000012,4.802023e-04,4.091679e-02,0.958591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,0,id17,0,id19,CGH,"0,1.45217985358423e-19,0.0591758520402779,0.78...",0.767,0.232,0.000,0.1164,-1,CGH,0.3,0.000000e+00,1.452180e-19,0.059176,7.831778e-01,1.456233e-01,0.012023
186,0,id17,0,id20,CGH,"0,4.5149591709111e-22,0.0356286063289017,0.608...",0.795,0.204,0.000,0.1021,-1,CGH,0.3,0.000000e+00,4.514959e-22,0.035629,6.081618e-01,2.444238e-01,0.111786
187,0,id18,0,id19,PC,"0.999488504078663,1.54522754683251e-18,0.00051...",0.000,1.000,0.000,0.5000,-1,PC,0.3,9.994885e-01,1.545228e-18,0.000511,4.009565e-13,4.171423e-27,0.000000
188,0,id18,0,id20,PC,"0.999488504078663,1.54522754683251e-18,0.00051...",0.000,1.000,0.000,0.5000,-1,PC,0.3,9.994885e-01,1.545228e-18,0.000511,4.009565e-13,4.171423e-27,0.000000


### Make a subset of rows where the combined likelihood proportion of 0 and 1 degrees is less than 80%

### For each row (ID pair) in the subset, calculate ERSA values using the helper functions and update the dataframe

### We can do this first by adding in a new string of comma-separated floats, and once it's confirmed working we'll just replace LIKELIHOOD_VECTOR and drop the extra columns

In [68]:

def calculate_ersa_props(model_file):

    #model_file = '/data100t1/home/grahame/projects/compadre/primus-ersa-v2/output/sim64/ersa/sim64_id2_id6_ersa.model'
    model_df = pd.read_csv(model_file, sep='\t', dtype={'maxlnl': float, 'degree_of_relatedness': int})
    model_df['maxlnl2'] = model_df['maxlnl'].apply(lambda x: math.exp(x))
    #model_df

    # every line is the pair we want, so we can skip the original step of checking that the IDs match

    model_df_2d = model_df[model_df['degree_of_relatedness'] == 2] 
    model_df_2d_sum = model_df_2d['maxlnl2'].sum()

    model_df_3d = model_df[model_df['degree_of_relatedness'] == 3] 
    model_df_3d_sum = model_df_3d['maxlnl2'].sum()

    model_df_4d = model_df[model_df['degree_of_relatedness'] == 4] 
    model_df_4d_sum = model_df_4d['maxlnl2'].sum()

    #model_df_un = model_df[(model_df['degree_of_relatedness'] >= 5) & (model_df['degree_of_relatedness'] <= 9)]
    model_df_un = model_df[model_df['degree_of_relatedness'] >= 5] 
    model_df_un_sum = model_df_un['maxlnl2'].sum()

    # re-proportion them 
    total_prop = model_df_2d_sum + model_df_3d_sum + model_df_4d_sum + model_df_un_sum
    model_df_2d_prop = model_df_2d_sum/total_prop
    model_df_3d_prop = model_df_3d_sum/total_prop
    model_df_4d_prop = model_df_4d_sum/total_prop
    model_df_un_prop = model_df_un_sum/total_prop

    return (model_df_2d_prop, model_df_3d_prop, model_df_4d_prop, model_df_un_prop)

#########################

matchfile = f'{data_dir}{prefix}.match'
mdf = pd.read_csv(matchfile, sep='\t', header=None)

#########################
# Isolate all potentially improve-able lines into their own df and iterate over them

df['p0-1'] = df['p0'] + df['p1']
subset = df[df['p0-1'] <= 0.8].reset_index(drop=True)
the_rest = df[df['p0-1'] > 0.8].reset_index(drop=True)

counter = 0

for x in range(len(subset)):

    id1 = subset.loc[x, 'IID1']
    id2 = subset.loc[x, 'IID2']

    # NEED TO CHECK IF THERE'S ANY PAIRWISE SHARING IN THE FIRST PLACE -- If not, ERSA won't help us, and we can skip it 
    match_subset = mdf[(mdf[0] == id1) & (mdf[1] == id2)]
    if len(match_subset) != 0:

        #print (f'{id1}, {id2}')

        ersa_outfile = f'{test_dir}ersa/{prefix}_{id1}_{id2}_ersa'
        ersa_command = f'python3 {ersa} --single_pair={id1}:{id2} --segment_files={matchfile} --model_output_file={ersa_outfile}.model --output_file={ersa_outfile}.out'
        os.system(ersa_command)

        # after ersa is run on that pair, do the same steps as in ersa_helper 
        output_model_file = ersa_outfile + '.model'

        # there is a chance that ERSA was not able to build results off a small # of segments, so we need to check if there's even results to use
        omf_df = pd.read_csv(output_model_file, sep='\t')
        if len(omf_df) != 0:
        
            ersa_props = calculate_ersa_props(output_model_file)

            # multiply the ERSA proportions by p0-2 to re-fit them back into the PRIMUS probability space
            prop02 = subset.loc[x, 'p0-1']
            ersa_props_updated = tuple(x * prop02 for x in ersa_props) # 2,3,4,UN

            # now we can update the subset values and ultimately LIKELIHOOD_VECTOR
            p0_old = subset.loc[x, 'p0']
            p1_old = subset.loc[x, 'p1']
            subset.loc[x, 'p2'] = ersa_props_updated[0]
            subset.loc[x, 'p3'] = ersa_props_updated[1]
            subset.loc[x, 'p4'] = ersa_props_updated[2]
            subset.loc[x, 'p5'] = ersa_props_updated[3]

            # update LIKELIHOOD_VECTOR
            subset.loc[x, 'LIKELIHOOD_VECTOR'] = f'{p0_old},{p1_old},{ersa_props_updated[0]},{ersa_props_updated[1]},{ersa_props_updated[2]},{ersa_props_updated[3]}'

            counter += 1

    # else:
    #     print (f'No segments in .match file: {id1} --- {id2}')


print (f'Updated {counter} lines using ERSA data')

'''Now we have to combine the 2'''

new_total_df = pd.concat([subset, the_rest]).reset_index(drop=True)
new_total_df = new_total_df.iloc[:, :-7]
new_total_df.to_csv('ersa_updated_likelihoods.csv', sep='\t', index=False)


Updated 57 lines using ERSA data


In [70]:
# drop newly added columns before writing to outfile for PRIMUS to read correctly 

new_total_df = new_total_df.iloc[:, :-7]
new_total_df

,FID1,IID1,FID2,IID2,MOST_LIKELY_REL,LIKELIHOOD_VECTOR,IBD0,IBD1,IBD2,PI_HAT,-1,POSSIBLE_RELS,LIKELIHOOD_CUTOFF
0,0,id1,0,id2,UN,"0,4.04568136411392e-51,1.2021878588324e-05,0.0...",1.000,0.000,0.000,0.0000,-1,UN,0.3
1,0,id1,0,id6,UN,"0,4.04568136411392e-51,1.2021878588324e-05,0.0...",1.000,0.000,0.000,0.0000,-1,UN,0.3
2,0,id1,0,id7,HAG,"2.09960757820347e-279,5.15980472780301e-05,0.9...",0.383,0.616,0.000,0.3082,-1,HAG,0.3
3,0,id1,0,id8,HAG,"0,1.71721654071259e-05,0.952398197490122,0.047...",0.445,0.554,0.000,0.2773,-1,HAG,0.3
4,0,id1,0,id9,HAG,"0,5.4802427340237e-06,0.886928084954092,0.1130...",0.483,0.516,0.000,0.2582,-1,HAG,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,0,id15,0,id17,FS,"1.17355090328876e-75,1,1.22554897627921e-17,1....",0.214,0.527,0.257,0.5213,-1,FS,0.3
186,0,id16,0,id17,FS,"4.26561403158653e-82,0.999999894290626,1.05709...",0.224,0.606,0.168,0.4721,-1,FS,0.3
187,0,id18,0,id19,PC,"0.999488504078663,1.54522754683251e-18,0.00051...",0.000,1.000,0.000,0.5000,-1,PC,0.3
188,0,id18,0,id20,PC,"0.999488504078663,1.54522754683251e-18,0.00051...",0.000,1.000,0.000,0.5000,-1,PC,0.3


In [71]:
new_total_df.to_csv('ersa_updated_likelihoods.csv', sep='\t', index=False)

In [3]:
answer = 0.0 + 4.04568136411392e-51 + 1.2731883053274535e-14 + 4.333455149898788e-12 + 5.2504925620957386e-09 + 0.9999999947451613
answer

1.0

### Working version that gets proportions correct

In [6]:
df = pd.read_csv('/data100t1/home/grahame/projects/compadre/primus-ersa-v2/example_data/test2/id3_id6_ersa.model', sep='\t')

# Filter out degrees over 9
df = df[(df['degree_of_relatedness'] >= 2) & (df['degree_of_relatedness'] <= 9)]

# Group by degree_of_relatedness and sum the maxlnl values
grouped = df.groupby('degree_of_relatedness')['maxlnl'].sum()

# Convert the summed likelihoods into proportions
proportions = grouped / grouped.sum()

### Maybe average out 5+ ? 

print(proportions)

degree_of_relatedness
2    0.239510
3    0.198634
4    0.146674
5    0.111812
6    0.090167
7    0.077347
8    0.069991
9    0.065865
Name: maxlnl, dtype: float64


In [19]:
model_file = '/data100t1/home/grahame/projects/compadre/primus-ersa-v2/example_data/test2/id2_id20_ersa.model'

model_df = pd.read_csv(model_file, sep='\t', dtype={'maxlnl': float, 'degree_of_relatedness': int})
model_df['maxlnl2'] = model_df['maxlnl'].apply(lambda x: math.exp(x))
subset = model_df[model_df['degree_of_relatedness'] <= 9]
subset2 = model_df[(model_df['degree_of_relatedness'] >= 2) & (model_df['degree_of_relatedness'] <= 9)]
maxlnl2sum = subset2['maxlnl2'].sum()
total_prop = maxlnl2sum/(subset['maxlnl2'].sum())


# every line is the pair we want, so we can skip the original step of checking that the IDs match

model_df_2d = model_df[model_df['degree_of_relatedness'] == 2] 
model_df_2d_sum = model_df_2d['maxlnl2'].sum()

model_df_3d = model_df[model_df['degree_of_relatedness'] == 3] 
model_df_3d_sum = model_df_3d['maxlnl2'].sum()

model_df_4d = model_df[model_df['degree_of_relatedness'] == 4] 
model_df_4d_sum = model_df_4d['maxlnl2'].sum()

model_df_un = model_df[(model_df['degree_of_relatedness'] >= 5) & (model_df['degree_of_relatedness'] <= 9)] 
model_df_un_sum = model_df_un['maxlnl2'].sum()

# re-proportion them 
total_sum = model_df_2d_sum + model_df_3d_sum + model_df_4d_sum + model_df_un_sum
print (f'Total proportion found: {total_prop}')
model_df_2d_prop = model_df_2d_sum/total_sum
model_df_3d_prop = model_df_3d_sum/total_sum
model_df_4d_prop = model_df_4d_sum/total_sum
model_df_un_prop = model_df_un_sum/total_sum

print (model_df_2d_prop, model_df_3d_prop, model_df_4d_prop, model_df_un_prop)


Total proportion found: 0.9999999997557558
7.183669034892495e-13 2.445051328555575e-10 2.962468370031077e-07 0.9999997035079395
